In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

In [3]:
import requests
import json
from pandas.io.json import json_normalize
from itertools import islice

In [622]:

# Envelope rond de provinciegrenzen doorzoeken. Alle objectids die daar binnen vallen downloaden en op een rijtje zetten.
envelope = '93003.99999999802+464251.5000000022+154328.6700000043+580999.9999999997' # Polygoon van NH trekt hij niet.
# In deze query wordt wegdeel_v gedownload. Dat is laag nummer 112. Aanpassen voor andere dataset.
laag = 'https://geoservices.noord-holland.nl/ags/rest/services/oi_op/oi_areaal/MapServer/97'

In [623]:
laaginfo = requests.get(laag+ '?f=pjson')
info = json_normalize(json.loads(laaginfo.content))
info_df = json_normalize(info.fields[0])
fieldsdf = info_df.name
fields = fieldsdf.tolist()
fields

['objectid',
 'shape',
 'doorvaarthoogteafg',
 'objectnaam',
 'volgnummer',
 'verwerkingsstatus',
 'ad_id',
 'objectbegintijd',
 'objecteindtijd']

In [624]:
query = laag +'/query?where=&text=&objectIds=&time=&geometry='+ envelope + '&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=&returnGeometry=false&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=true&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&f=pjson'
r =  requests.get(query, verify=False)
df = json_normalize(json.loads(r.content))
df.objectIds[0][0]
objectIds_list = df.objectIds[0]

/home/dataiku/dss/code-envs/python/Gerda/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geoservices.noord-holland.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [625]:
# Velden kiezen die je wilt downloaden en in het juiste format zetten of alles houden.
#fields = ['SHAPE','WEGBEHSRT','WEGNUMMER','WEGDEELLTR','HECTO_LTTR','BAANSUBSRT','RPE_CODE', 'ADMRICHTNG', 'RIJRICHTNG','STT_NAAM','WPSNAAMNEN','OBJECTID_1','OBJECTID']
fieldsstring = ''
for f in range(0,len(fields)):
    field = fields[f]
    if fieldsstring == '':
        fieldsstring = field
    else:
        fieldsstring = fieldsstring + '%2C+' + field

In [626]:
# objectIds opsplitsen in chunks
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())
n = 100 # grootte van chunks
objectIds_chunks = list(chunk(objectIds_list, n))
aantalchunks = len(objectIds_chunks)

In [627]:
# Areaaldata downloaden per chunk
#df_PNH = pd.DataFrame()
for i in range(0,aantalchunks):
    chunk = objectIds_chunks[i]
    objectids = ''
    for j in range(0,len(chunk)):
        objectid = str(chunk[j])
        if objectids == '':
            objectids = objectid
        else:
            objectids = objectids + '%2C+' + objectid
    query2=laag+'/query?where=&text=&objectIds='+ objectids +'&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields='+ fieldsstring + '&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&f=pjson'
    r =  requests.get(query2, verify=False)
    df2 = json_normalize(json.loads(r.content))
    data = json_normalize(df2.features[0])
    data['ID'] = 97
    df_PNH = df_PNH.append(data)
    

/home/dataiku/dss/code-envs/python/Gerda/lib/python3.6/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geoservices.noord-holland.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [628]:
len(df_PNH)

396392

In [629]:
df_PNH.tail()

,ID,attributes.aantalgroepen,attributes.aantaloverspanning,attributes.aantalwindingen,attributes.ad_id,attributes.afstandverharding,attributes.afvalwatertype,attributes.automaatkoppeling,attributes.automaatnummer,attributes.bedrijfsspanning,attributes.beheerder,attributes.beheerobjectsubtyp,attributes.besluitnummer,attributes.besteknummer,attributes.bgtplustype,attributes.bodemhoogte,attributes.boomsituering,attributes.boomsoort,attributes.bordnummer,attributes.bouwjaar,attributes.bovenkantslib,attributes.bprtypebord,attributes.bronhouder,attributes.buis,attributes.cat_waterloop,attributes.codekabel,attributes.conditiescore_opm,attributes.contractnumonderh,attributes.dataleverancier,attributes.datumplaatsing,attributes.deeltrajectnummer,attributes.detectienummer,attributes.diameter,attributes.diametercm,attributes.diametermm,attributes.dieptelegging,attributes.doorrijhoogterijst,attributes.doorrijhoogterijst_1,attributes.doorrijhoogterijst_2,attributes.doorrijhoogterijst_3,attributes.doorrijhoogterijst_4,attributes.doorrijhoogterijst_5,attributes.doorvaarthoogte1,attributes.doorvaarthoogte2,attributes.doorvaarthoogte3,attributes.doorvaarthoogte4,attributes.doorvaarthoogte5,attributes.doorvaarthoogteafg,attributes.eigenaar,attributes.fabrikanttypecode,attributes.faunasoort,attributes.foto,attributes.foto2,attributes.fysiekvoorkomen,attributes.geonauwkeurigheidxy,attributes.gewenstediepte,attributes.gisib_id,attributes.gsmnummer,attributes.halte,attributes.haltenummer,attributes.hectometer,attributes.hoogte,attributes.hoogtebovennap,attributes.huidigestatus,attributes.idcode,attributes.imkl_innetwork,attributes.indicatieveligging,attributes.innetwerk,attributes.kastnummer,attributes.kastovl,attributes.kastvri,attributes.kerendehoogte,attributes.kilometrering,attributes.kp_code,attributes.kwaliteitsniveau,attributes.kwbreedte,attributes.kwhoogte,attributes.kwlengte,attributes.lantaarnnummer,attributes.lengte,attributes.lengtelinkeroever,attributes.lengterechteroever,attributes.lengteuithouder1,attributes.lengteuithouder2,attributes.lichtkarakter,attributes.lichtpuntnummer,attributes.locatie,attributes.mastnummer,attributes.materiaaltype,attributes.montagewijze,attributes.naam,attributes.naar,attributes.ndw_code,attributes.nenbeheerobject,attributes.nominalespanning,attributes.nummer,attributes.nummergekoppeldeau,attributes.objectbegintijd,attributes.objectcode,attributes.objecteindtijd,attributes.objectid,attributes.objectnaam,attributes.oevervak,attributes.omschrijving,attributes.omtrek,attributes.onderhouder,attributes.onderkantslib,attributes.openbaar,attributes.opmerking,attributes.oppervlakte,attributes.paal,attributes.paal2,attributes.paalnummer,attributes.plaats,attributes.postcode_plaats,attributes.priovoorzieningen,attributes.reflectieklasse,attributes.relatievehoogteligging,attributes.rvvtypebord,attributes.serienummer,attributes.simnummer,attributes.snoeifase,attributes.st_area(shape),attributes.st_length(shape),attributes.st_perimeter(shape),attributes.tekst,attributes.telpuntcode,attributes.topcode,attributes.traject,attributes.traject2,attributes.typeplaaginvasiesoort,attributes.typeplaagsoort,attributes.typespec,attributes.vaarwegdeeltraject,attributes.van,attributes.verticale_positie,attributes.verwerkingsstatus,attributes.volgnummer,attributes.waterdiepte,attributes.wegnaam,attributes.wegnr,attributes.wegnummer,attributes.worteldoek_scherm,attributes.xcoordinaat,attributes.ycoordinaat,attributes.zijde,attributes.zijweg,attributes.zitmeubel,attributes.zonnepaneel,geometry.paths,geometry.points,geometry.rings,geometry.x,geometry.y
0,97,NaN,NaN,NaN,AD.8D5CB1AB0EFC418592FE00F6629AA1D4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,439 m (hoogte = 2,039 + NAP; waterpeil 0,40 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [0]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("mydataset")
mydataset_df = mydataset.get_dataframe()